In [ ]:
import pandas as pd
import numpy as np

dir_path = "/home/thomas/Dokumente/Master/Master_Thesis/results/BS_Laura/channel_space/ft_['Slope']/clf_LDA/prune_ch_sma_False/sel_hrf_None"
df = pd.read_csv(dir_path + "/ws_accuracy_results_ft_rf_int_03_sp_3.csv")

df['Mean_Accuracy'] = pd.to_numeric(df['Mean_Accuracy'], errors='coerce')

subjects = sorted(df['Subject'].unique())
subsets = sorted(df['Subset'].unique())

results = {}

for subset in subsets:
    print("")
    print(subset)
    print("")
    subset_data = df[df['Subset'] == subset]
    row = {}
    for subject in subjects:
        print(subject)
        subj_data = subset_data[subset_data['Subject'] == subject]
        
        long_val = subj_data[subj_data['Condition'] == 'long']['Mean_Accuracy']
        ss_val = subj_data[subj_data['Condition'] == 'long_ss_mean']['Mean_Accuracy']

        print("long_val")
        print(long_val)
        print("ss_val")
        print(ss_val)

        
        long_str = f"{round(long_val.values[0]*100):.0f}" if not long_val.empty else ""
        ss_str = f"{round(ss_val.values[0]*100):.0f}" if not np.isnan(ss_val.values[0]) else ""
        
        if long_str and ss_str:
            val = f"{long_str}/{ss_str}"
        elif long_str:
            val = long_str
        elif ss_str:
            val = ss_str
        else:
            val = ""
        
        row[subject] = val
    
    results[subset] = row

# Convert to DataFrame
table = pd.DataFrame(results).T[subjects]  # Transpose so subsets are rows
table.index.name = 'Subset'

# Function to extract A and B from "A/B"
def extract_scores(cell):
    if isinstance(cell, str) and '/' in cell:
        a, b = cell.split('/')
        return int(a), int(b)
    elif isinstance(cell, str) and cell.isdigit():
        return int(cell), None
    return None, None

# Compute mean A and mean B for each row
mean_col = []
for idx, row in table.iterrows():
    a_values = []
    b_values = []
    for cell in row:
        a, b = extract_scores(cell)
        if a is not None:
            a_values.append(a)
        if b is not None:
            b_values.append(b)
    
    mean_a = round(np.mean(a_values), 1) if a_values else ""
    mean_b = round(np.mean(b_values), 1) if b_values else ""
    
    if mean_a and mean_b:
        mean_col.append(f"{mean_a}/{mean_b}")
    elif mean_a:
        mean_col.append(f"{mean_a}")
    elif mean_b:
        mean_col.append(f"{mean_b}")
    else:
        mean_col.append("")

# Assign mean column to table
table['¯x'] = mean_col

# Print final table
#print(table.to_string())
for idx, row in table.iterrows():
    row_values = [str(row[subj]) for subj in subjects] + [row['¯x']]
    row_str = f"{idx} & " + " & ".join(row_values) + " \\\\"
    print(row_str)



full

sub-577
long_val
36    0.8
Name: Mean_Accuracy, dtype: float64
ss_val
84    0.8
Name: Mean_Accuracy, dtype: float64
sub-580
long_val
37    0.6444
Name: Mean_Accuracy, dtype: float64
ss_val
85    0.6667
Name: Mean_Accuracy, dtype: float64
sub-586
long_val
38    0.7222
Name: Mean_Accuracy, dtype: float64
ss_val
86    0.8111
Name: Mean_Accuracy, dtype: float64
sub-587
long_val
39    0.8778
Name: Mean_Accuracy, dtype: float64
ss_val
87    0.7444
Name: Mean_Accuracy, dtype: float64
sub-592
long_val
40    0.6333
Name: Mean_Accuracy, dtype: float64
ss_val
88    0.5444
Name: Mean_Accuracy, dtype: float64
sub-613
long_val
41    0.7
Name: Mean_Accuracy, dtype: float64
ss_val
89    0.5444
Name: Mean_Accuracy, dtype: float64
sub-618
long_val
42    0.8
Name: Mean_Accuracy, dtype: float64
ss_val
90    0.8222
Name: Mean_Accuracy, dtype: float64
sub-619
long_val
43    0.7444
Name: Mean_Accuracy, dtype: float64
ss_val
91    0.6778
Name: Mean_Accuracy, dtype: float64
sub-621
long_val
44    0.6222

### with stars for significance

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import multipletests

# --- Parameter ---
dir_path = "/home/thomas/Dokumente/Master/Master_Thesis/results/BS_Laura/channel_space/ft_['Slope']/clf_LDA/prune_ch_sma_True/sel_hrf_None"
csv_file = dir_path + "/ws_accuracy_results_ft_rf_int_03_sp_3.csv"

# Subset-Reihenfolge festlegen
subset_order = ["subset_3", "subset_2", "subset_1", "full"]

# --- Daten laden ---
df = pd.read_csv(csv_file)
df['Mean_Accuracy'] = pd.to_numeric(df['Mean_Accuracy'], errors='coerce')

subjects = sorted(df['Subject'].unique())
subsets = [s for s in subset_order if s in df['Subset'].unique()]  # nur vorhandene verwenden

# --- Datenstruktur: dict[subset][condition] = array von Werten ---
data_dict = {}
for subset in subsets:
    data_dict[subset] = {}
    for cond in ['long', 'long_ss_mean']:
        cond_vals = []
        for subj in subjects:
            val = df[(df['Subset'] == subset) &
                     (df['Subject'] == subj) &
                     (df['Condition'] == cond)]['Mean_Accuracy']
            if not val.empty:
                cond_vals.append(val.values[0] * 100)
        data_dict[subset][cond] = np.array(cond_vals)


pvals_star = []
star_pairs = []
for i in range(1, len(subsets)):
    for cond in ['long', 'long_ss_mean']:
        if len(data_dict[subsets[i]][cond]) > 0 and len(data_dict[subsets[i-1]][cond]) > 0:
            t, p = ttest_rel(data_dict[subsets[i]][cond], data_dict[subsets[i-1]][cond])
            pvals_star.append(p)
            star_pairs.append((subsets[i], cond))

star_uncorr = {pair: p < 0.05 for pair, p in zip(star_pairs, pvals_star)}  # uncorrected
if pvals_star:
    reject_star, pvals_corr_star, _, _ = multipletests(pvals_star, alpha=0.05, method='holm')
    star_corr = {pair: sig for pair, sig in zip(star_pairs, reject_star)}
else:
    star_corr, star_uncorr = {}, {}

pvals_dagger = []
dagger_pairs = []
for subset in subsets:
    if len(data_dict[subset]['long']) > 0 and len(data_dict[subset]['long_ss_mean']) > 0:
        t, p = ttest_rel(data_dict[subset]['long'], data_dict[subset]['long_ss_mean'])
        pvals_dagger.append(p)
        dagger_pairs.append(subset)

dagger_uncorr = {subset: p < 0.05 for subset, p in zip(dagger_pairs, pvals_dagger)}
if pvals_dagger:
    reject_dagger, pvals_corr_dagger, _, _ = multipletests(pvals_dagger, alpha=0.05, method='holm')
    dagger_corr = {subset: sig for subset, sig in zip(dagger_pairs, reject_dagger)}
else:
    dagger_corr, dagger_uncorr = {}, {}

results = {}

# --- 3) Build base results table ---
results = {}
for subset in subsets:
    row = {}
    for subj in subjects:
        long_val = df[(df['Subset'] == subset) &
                      (df['Subject'] == subj) &
                      (df['Condition'] == 'long')]['Mean_Accuracy']
        ss_val = df[(df['Subset'] == subset) &
                    (df['Subject'] == subj) &
                    (df['Condition'] == 'long_ss_mean')]['Mean_Accuracy']

        long_str = f"{round(long_val.values[0]*100):.0f}" if not long_val.empty else ""
        ss_str = f"{round(ss_val.values[0]*100):.0f}" if not ss_val.empty and not np.isnan(ss_val.values) else ""
        
        if long_str and ss_str:
            val = f"{long_str}/{ss_str}"
        elif long_str:
            val = long_str
        elif ss_str:
            val = ss_str
        else:
            val = ""
        row[subj] = val

    # Mean values for ¯x
    mean_a = np.mean(data_dict[subset]['long']) if len(data_dict[subset]['long']) > 0 else np.nan
    mean_b = np.mean(data_dict[subset]['long_ss_mean']) if len(data_dict[subset]['long_ss_mean']) > 0 else np.nan
    if not np.isnan(mean_a) and not np.isnan(mean_b):
        mean_str = f"{mean_a:.1f}/{mean_b:.1f}"
    elif not np.isnan(mean_a):
        mean_str = f"{mean_a:.1f}"
    elif not np.isnan(mean_b):
        mean_str = f"{mean_b:.1f}"
    else:
        mean_str = ""
    row['¯x'] = mean_str

    results[subset] = row

# --- 4) Markierungen ---
for subset in subsets:
    for cond in ['long', 'long_ss_mean']:
        if (subset, cond) in star_corr:
            if star_corr[(subset, cond)]:
                star_symbol = "**"
            elif star_uncorr[(subset, cond)]:
                star_symbol = "*"
            else:
                star_symbol = ""
            if star_symbol:
                if cond == 'long':
                    parts = results[subset]['¯x'].split('/')
                    parts[0] += star_symbol
                    results[subset]['¯x'] = "/".join(parts) if len(parts) == 2 else parts[0]
                else:
                    if '/' in results[subset]['¯x']:
                        parts = results[subset]['¯x'].split('/')
                        parts[1] += star_symbol
                        results[subset]['¯x'] = "/".join(parts)

    #if subset in dagger_corr:
    #    if dagger_corr[subset]:
    #        results[subset]['¯x'] += "††"
    #    elif dagger_uncorr[subset]:
    #        results[subset]['¯x'] += "†"

subset_names = {"subset_3": 'Sparse', "subset_2": 'Low-Density', "subset_1": 'Mid-Density',"full": 'High-Density',}

subset_format = {
    "subset_3": (r"\rowcolor{subset3}", "Sparse"),
    "subset_2": (r"\rowcolor{subset2}", "Low-Density"),
    "subset_2fp": (r"\rowcolor{subset2fp}", "Low-Density FP GLM"),
    "subset_1": (r"\rowcolor{subset1}", "Mid-Density"),
    "subset_1fp": (r"\rowcolor{subset1fp}", "Mid-Density FP GLM"),
    "full": (r"\rowcolor{full}", "High-Density"),
    "fullfp": (r"\rowcolor{fullfp}", "High-Density FP GLM"),
}

# --- 5) LaTeX-Ausgabe ---
for idx, subset in enumerate(subsets):
    rowcolor, label = subset_format.get(subset, ("", subset))
    row_values = [results[subset][subj] for subj in subjects] + [f"\\textbf{{{results[subset]['¯x']}}}"]

    if idx == 0:
        print(f"& {rowcolor}  {label}  & " + " & ".join(row_values) + r" \\")
    else:
        print(f" & {rowcolor} {label}  & " + " & ".join(row_values) + r" \\")

& \rowcolor{subset3}  Sparse  & 39 & 72 & 73 & 96 & 67 & 77 & 58 & 73 & 67 & 88 & 94 & 66 & \textbf{72.4} \\
 & \rowcolor{subset2} Low-Density  & 43/63 & 68/80 & 87/89 & 91/91 & 68/60 & 69/72 & 52/67 & 70/57 & 76/68 & 84/96 & 93/90 & 62/69 & \textbf{71.9/75.1} \\
 & \rowcolor{subset1} Mid-Density  & 48/62 & 79/83 & 84/91 & 93/91 & 70/59 & 71/80 & 54/73 & 66/63 & 60/66 & 83/98 & 93/90 & 67/64 & \textbf{72.4/76.8} \\
 & \rowcolor{full} High-Density  & 79/84 & 79/81 & 87/86 & 92/91 & 70/62 & 80/83 & 81/86 & 67/63 & 69/70 & 96/100 & 97/93 & 63/60 & \textbf{79.9*/80.0} \\
